In [38]:
from OpenSSL import crypto

TYPE_RSA = crypto.TYPE_RSA
TYPE_DSA = crypto.TYPE_DSA

def createKeyPair(type, bits):
    pkey = crypto.PKey()
    pkey.generate_key(type, bits)
    return pkey


In [39]:
from OpenSSL import crypto

def createCertRequest(pkey, digest="md5", **name):
    req = crypto.X509Req()
    subj = req.get_subject()

    for (key,value) in name.items():
        setattr(subj, key, value)

    req.set_pubkey(pkey)
    req.sign(pkey, digest)
    return req

In [42]:
from OpenSSL import SSL,crypto

def createCertificate(req,issuerCert, issuerKey, serial,notBefore, notAfter, digest="md5"):
    cert = crypto.X509()
    cert.set_serial_number(serial)
    cert.gmtime_adj_notBefore(notBefore)
    cert.gmtime_adj_notAfter(notAfter)
    cert.set_issuer(issuerCert.get_subject())
    cert.set_subject(req.get_subject())
    cert.set_pubkey(req.get_pubkey())
    cert.sign(issuerKey, digest)
    return cert

In [51]:
from OpenSSL import crypto

cakey = createKeyPair(TYPE_RSA, 1024)
careq = createCertRequest(cakey, CN='Certificate Authority')
cacert = createCertificate(careq,careq, cakey, 0, 0, 60*60*24*365) # um ano de validade
open('CA.pkey', 'wb').write(crypto.dump_privatekey(crypto.FILETYPE_PEM, cakey))
open('CA.cert', 'wb').write(crypto.dump_certificate(crypto.FILETYPE_PEM, cacert))
for (fname, cname) in [('client', 'Simple Client'), ('server', 'Simple Server')]:
    pkey = createKeyPair(TYPE_RSA, 1024)
    req = createCertRequest(pkey, CN=cname)
    cert = createCertificate(req,cacert, cakey, 1, 0, 60*60*24*365) # um ano de validade
    open('%s.pkey' % (fname,), 'wb').write(crypto.dump_privatekey(crypto.FILETYPE_PEM, pkey))
    open('%s.cert' % (fname,), 'wb').write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert))

